In [22]:
from blendshape_info import BLENDSHAPE_MODEL_LANDMARKS_SUBSET, BLENDSHAPE_NAMES, parse_prototxt

path = 'face_blendshapes_in_landmarks.prototxt'
test_lmks_in = parse_prototxt(path)[:, :2]
path = 'face_blendshapes_out.prototxt'
test_blshps_out = parse_prototxt(path)

In [298]:
import onnx
from onnx import helper
import onnxruntime as rt

onnx_path = './face_blendshapes.onnx'
modified_model = onnx.load(onnx_path)
intermediate_layer_value_info = helper.ValueInfoProto()

v = ['concat']
v = ['model_1/tf.__operators__.getitem_2/strided_slice3']
# v = ['model_1/tf.compat.v1.norm/norm/mul']
# [(i, n) for i, n in enumerate(modified_model.graph.node) if v[0] in str(n)]
# v = ['model_1/tf.compat.v1.norm/norm/Sqrt']
v = ['model_1/GhumMarkerPoserMlpMixerGeneral/conv2d/BiasAdd;model_1/GhumMarkerPoserMlpMixerGeneral/conv2d/Conv2D;model_1/GhumMarkerPoserMlpMixerGeneral/conv2d/BiasAdd/ReadVariableOp']
v = ['model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/input_tokens_embedding/BiasAdd;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_3/mlp_channel_mixing/Mlp_2/Conv2D;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/input_tokens_embedding/Conv2D;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/input_tokens_embedding/BiasAdd/ReadVariableOp']

v = ['model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/AddExtraTokens/concat']
v = ['model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/layer_norm1/moments/SquaredDifference__43:0']
v = ['model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Relu;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_3/mlp_token_mixing/Mlp_1/Conv2D;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Conv2D;model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd/ReadVariableOp__45:0']
# v = ['model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/layer_norm1/batchnorm/add_1']
v = '''
name: model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_3/residual_channels/add
'''
if not isinstance(v, list):
    v = [(v.split("name: ")[1] if len(v.split("name: ")) > 1 else v).strip()]

modified_model_path = '/tmp/modified.onnx'
intermediate_layer_value_info = helper.ValueInfoProto()
intermediate_layer_value_info.name = v[0]
modified_model.graph.output.append(intermediate_layer_value_info)
onnx.save(modified_model, modified_model_path)

modified_model = onnx.load(modified_model_path)

providers = ['CPUExecutionProvider']
m = rt.InferenceSession(modified_model_path, providers=providers)
input_names = [n.name for n in modified_model.graph.input]

xx = test_lmks_in[None, BLENDSHAPE_MODEL_LANDMARKS_SUBSET ]
outs = m.run(v,  {input_names[0]: xx})
outs[0].shape

(1, 64, 1, 97)

In [299]:
# # Conv graph nodes:
# ' '.join([str(i) for i in range(len(modified_model.graph.node)) if 'Conv' in str(modified_model.graph.node[i])])

In [312]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from onnx.numpy_helper import to_array
import numpy as np


def get_model_weight(onnx_model, name):
    for _ in onnx_model.graph.initializer:
        if _.name == name:
            break
    return _

def conv_node_to_w_b(onnx_model, node):
    w_node = node.input[1]
    w = to_array(get_model_weight(onnx_model, w_node))
    assert len(w.shape) == 4, w.shape
    b_node = node.input[2]
    b = to_array(get_model_weight(onnx_model, b_node))
    assert len(b.shape) == 1, b.shape
    w, b = [torch.from_numpy(_).float() for _ in [w, b]]
    return w, b

def get_layernorm_weight(onnx_model, mixer_block_idx, norm_idx):
    search_str = f'model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_{mixer_block_idx}/layer_norm{norm_idx}/batchnorm/mul'
    fold_op_name = [n for n in onnx_model.graph.node if search_str == n.output[0]][0].input[1]
    return get_model_weight(onnx_model, fold_op_name)

def get_conv_layer_weight_bias(onnx_model, mixer_block_idx, is_token_mixer, mlp_idx):
    # print(mixer_block_idx, is_token_mixer, mlp_idx)
    assert mlp_idx in (1, 2)
    assert mixer_block_idx in (0, 1, 2, 3)
    search_str = '''
    model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Relu;
    model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd;
    model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_3/mlp_token_mixing/Mlp_1/Conv2D;
    model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Conv2D;
    model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd/ReadVariableOp
    '''
    if mixer_block_idx == 3:
        search_str = '''
        model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Relu;
        model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd;
        model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Conv2D;
        model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/BiasAdd/ReadVariableOp
        '''

    
    search_str = search_str.replace("\n", "").replace(" ", "").strip()

    search_str = search_str.replace("MixerBlock_0", f'MixerBlock_{mixer_block_idx}')
    if mlp_idx == 2:
        replace_str = "model_1/GhumMarkerPoserMlpMixerGeneral/MLPMixer/MixerBlock_0/mlp_token_mixing/Mlp_1/Relu;"
        replace_str = replace_str.replace("MixerBlock_0", f'MixerBlock_{mixer_block_idx}')
        search_str = search_str.replace(replace_str, "")
    search_str = search_str.replace("Mlp_1", f'Mlp_{mlp_idx}')
    if not is_token_mixer:
        search_str = search_str.replace("mlp_token_mixing", "mlp_channel_mixing")
    # print(search_str)
    ii, node = [(i, n) for i, n in enumerate(onnx_model.graph.node) if search_str == n.output[0]][0]
    # print(ii)
    w, b = conv_node_to_w_b(onnx_model, node)
    mlpname = 'mlp_token_mixing' if is_token_mixer else 'mlp_channel_mixing'
    idx = 0 if mlp_idx == 1 else 3
    # print(w.shape, b.shape)
    # print(w[:2, :2, 0, 0], b[:2]    )
    return {f"mlpmixer_blocks.{mixer_block_idx}.{mlpname}.{idx}.weight": w, f"mlpmixer_blocks.{mixer_block_idx}.{mlpname}.{idx}.bias": b}


def get_state_dict_mlp_mixer_layer(onnx_model, mixer_block_idx):
    state_dict = {}
    norm1_weight = get_layernorm_weight(onnx_model, mixer_block_idx, 1)
    state_dict.update({f"mlpmixer_blocks.{mixer_block_idx}.norm1.weight": torch.from_numpy(to_array(norm1_weight)).float()})
    state_dict.update(get_conv_layer_weight_bias(onnx_model, mixer_block_idx, True, 1))
    state_dict.update(get_conv_layer_weight_bias(onnx_model, mixer_block_idx, True, 2))
    norm2_weight = get_layernorm_weight(modified_model, mixer_block_idx, 2)
    state_dict.update({f"mlpmixer_blocks.{mixer_block_idx}.norm2.weight": torch.from_numpy(to_array(norm2_weight)).float()})
    state_dict.update(get_conv_layer_weight_bias(onnx_model, mixer_block_idx, False, 1))
    state_dict.update(get_conv_layer_weight_bias(onnx_model, mixer_block_idx, False, 2))
    return state_dict


def get_state_dict(onnx_model):
    w, b = conv_node_to_w_b(onnx_model, onnx_model.graph.node[11])
    state_dict = {"conv1.weight": w, "conv1.bias": b}
    w, b = conv_node_to_w_b(onnx_model, onnx_model.graph.node[13])
    state_dict.update({"conv2.weight": w, "conv2.bias": b})
    extra_token = get_model_weight(onnx_model, 'const_fold_opt__350')
    state_dict.update({"extra_token": torch.from_numpy(to_array(extra_token)).float()})

    # MLP Mixer layers
    # layer 0
    state_dict.update(get_state_dict_mlp_mixer_layer(onnx_model, 0))
    # layer 1
    state_dict.update(get_state_dict_mlp_mixer_layer(onnx_model, 1))
    # layer 2
    state_dict.update(get_state_dict_mlp_mixer_layer(onnx_model, 2))
    # layer 3
    state_dict.update(get_state_dict_mlp_mixer_layer(onnx_model, 3))

    return state_dict

get_state_dict(modified_model).keys()

dict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'extra_token', 'mlpmixer_blocks.0.norm1.weight', 'mlpmixer_blocks.0.mlp_token_mixing.0.weight', 'mlpmixer_blocks.0.mlp_token_mixing.0.bias', 'mlpmixer_blocks.0.mlp_token_mixing.3.weight', 'mlpmixer_blocks.0.mlp_token_mixing.3.bias', 'mlpmixer_blocks.0.norm2.weight', 'mlpmixer_blocks.0.mlp_channel_mixing.0.weight', 'mlpmixer_blocks.0.mlp_channel_mixing.0.bias', 'mlpmixer_blocks.0.mlp_channel_mixing.3.weight', 'mlpmixer_blocks.0.mlp_channel_mixing.3.bias', 'mlpmixer_blocks.1.norm1.weight', 'mlpmixer_blocks.1.mlp_token_mixing.0.weight', 'mlpmixer_blocks.1.mlp_token_mixing.0.bias', 'mlpmixer_blocks.1.mlp_token_mixing.3.weight', 'mlpmixer_blocks.1.mlp_token_mixing.3.bias', 'mlpmixer_blocks.1.norm2.weight', 'mlpmixer_blocks.1.mlp_channel_mixing.0.weight', 'mlpmixer_blocks.1.mlp_channel_mixing.0.bias', 'mlpmixer_blocks.1.mlp_channel_mixing.3.weight', 'mlpmixer_blocks.1.mlp_channel_mixing.3.bias', 'mlpmixer_blocks.2.norm1.w

In [313]:
class MLPMixerLayer(nn.Module):
    def __init__(self, in_dim, num_patches, hidden_units_mlp1, hidden_units_mlp2, dropout_rate=0.0, eps1=0.0000010132789611816406, eps2=0.0000010132789611816406):
        super().__init__()
        self.mlp_token_mixing = nn.Sequential(
            nn.Conv2d(num_patches, hidden_units_mlp1, 1),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(hidden_units_mlp1, num_patches, 1)
        )
        self.mlp_channel_mixing = nn.Sequential(
            nn.Conv2d(in_dim, hidden_units_mlp2, 1),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(hidden_units_mlp2, in_dim, 1)
        )
        self.norm1 = nn.LayerNorm(in_dim, bias=False, elementwise_affine=True, eps=eps1)
        self.norm2 = nn.LayerNorm(in_dim, bias=False, elementwise_affine=True, eps=eps2)

    def forward(self, x):
        x_1 = self.norm1(x)
        mlp1_outputs = self.mlp_token_mixing(x_1)
        x = x + mlp1_outputs
        x_2 = self.norm2(x)
        mlp2_outputs = self.mlp_channel_mixing(x_2.permute(0, 3, 2, 1))
        x = x + mlp2_outputs.permute(0, 3, 2, 1)
        return x


class MLPMixer(nn.Module):
    def __init__(self, in_dim=64, num_patches=97, hidden_units_mlp1=384, hidden_units_mlp2=256, num_blocks=4, dropout_rate=0.0):
        super().__init__()
        self.conv1 = nn.Conv2d(146, 96, kernel_size=1)
        self.conv2 = nn.Conv2d(2, 64, kernel_size=1)
        self.extra_token = nn.Parameter(torch.randn(1, 64, 1, 1), requires_grad=True)
        self.mlpmixer_blocks = nn.Sequential(
            *[MLPMixerLayer(in_dim, num_patches, hidden_units_mlp1, hidden_units_mlp2, dropout_rate) for _ in range(num_blocks)]
        )

    def forward(self, x):
        x = x - x.mean(1, keepdim=True)
        x = x / x.norm(dim=2, keepdim=True).mean(1, keepdim=True)
        x = x.unsqueeze(-2) * 0.5
        x = self.conv1(x)
        x = x.permute(0, 3, 2, 1)
        x = self.conv2(x)
        x = torch.cat([self.extra_token, x], dim=3)
        x = x.permute(0, 3, 2, 1)
        x = self.mlpmixer_blocks(x)
        return x.permute(0, 3, 2, 1)

m = MLPMixer()
print(m.state_dict().keys())
m.load_state_dict(get_state_dict(modified_model))
xx_tch = torch.from_numpy(xx)
tch_out = m(xx_tch).detach().numpy()
onnx_out = outs[0]
# print(tch_out[:5])
# print(onnx_out[:5])
print("torch:", tch_out.shape)
print("onnx :", onnx_out.shape)
np.abs(tch_out - onnx_out).max()

odict_keys(['extra_token', 'conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'mlpmixer_blocks.0.mlp_token_mixing.0.weight', 'mlpmixer_blocks.0.mlp_token_mixing.0.bias', 'mlpmixer_blocks.0.mlp_token_mixing.3.weight', 'mlpmixer_blocks.0.mlp_token_mixing.3.bias', 'mlpmixer_blocks.0.mlp_channel_mixing.0.weight', 'mlpmixer_blocks.0.mlp_channel_mixing.0.bias', 'mlpmixer_blocks.0.mlp_channel_mixing.3.weight', 'mlpmixer_blocks.0.mlp_channel_mixing.3.bias', 'mlpmixer_blocks.0.norm1.weight', 'mlpmixer_blocks.0.norm2.weight', 'mlpmixer_blocks.1.mlp_token_mixing.0.weight', 'mlpmixer_blocks.1.mlp_token_mixing.0.bias', 'mlpmixer_blocks.1.mlp_token_mixing.3.weight', 'mlpmixer_blocks.1.mlp_token_mixing.3.bias', 'mlpmixer_blocks.1.mlp_channel_mixing.0.weight', 'mlpmixer_blocks.1.mlp_channel_mixing.0.bias', 'mlpmixer_blocks.1.mlp_channel_mixing.3.weight', 'mlpmixer_blocks.1.mlp_channel_mixing.3.bias', 'mlpmixer_blocks.1.norm1.weight', 'mlpmixer_blocks.1.norm2.weight', 'mlpmixer_blocks.2.mlp_to

2.1457672e-05

In [293]:
class MLPMixerLayer(nn.Module):
    def __init__(self, num_patches, hidden_units, dropout_rate):
        super().__init__()

        self.mlp1 = nn.Sequential(
            nn.Linear(num_patches, num_patches),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(num_patches, num_patches)
        )

        self.mlp2 = nn.Sequential(
            nn.Linear(num_patches, num_patches),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(num_patches, hidden_units)
        )

        self.norm1 = nn.LayerNorm(num_patches)
        self.norm2 = nn.LayerNorm(hidden_units)

    def forward(self, x):
        # Apply layer normalization
        x = self.norm1(x)
        return x

        # Transpose inputs from [batch_size, num_patches, hidden_units] to [batch_size, hidden_units, num_patches]
        x_channels = x.transpose(1, 2)

        # Apply mlp1 on each channel independently
        mlp1_outputs = self.mlp1(x_channels)

        # Transpose mlp1_outputs from [batch_size, hidden_units, num_patches] to [batch_size, num_patches, hidden_units]
        mlp1_outputs = mlp1_outputs.transpose(1, 2)

        # Add skip connection
        x = mlp1_outputs + x

        # Apply layer normalization
        x = self.norm2(x)

        # Apply mlp2 on each patch independently
        mlp2_outputs = self.mlp2(x)

        # Add skip connection
        x = mlp2_outputs + x

        return x

In [21]:
if 0:
    class MLPMixerLayer(layers.Layer):
        def __init__(self, num_patches, hidden_units, dropout_rate, *args, **kwargs):
            super().__init__(*args, **kwargs)

            self.mlp1 = keras.Sequential(
                [
                    layers.Dense(units=num_patches, activation="gelu"),
                    layers.Dense(units=num_patches),
                    layers.Dropout(rate=dropout_rate),
                ]
            )
            self.mlp2 = keras.Sequential(
                [
                    layers.Dense(units=num_patches, activation="gelu"),
                    layers.Dense(units=hidden_units),
                    layers.Dropout(rate=dropout_rate),
                ]
            )
            self.normalize = layers.LayerNormalization(epsilon=1e-6)

        def build(self, input_shape):
            return super().build(input_shape)

        def call(self, inputs):
            # Apply layer normalization.
            x = self.normalize(inputs)
            # Transpose inputs from [num_batches, num_patches, hidden_units] to [num_batches, hidden_units, num_patches].
            x_channels = keras.ops.transpose(x, axes=(0, 2, 1))
            # Apply mlp1 on each channel independently.
            mlp1_outputs = self.mlp1(x_channels)
            # Transpose mlp1_outputs from [num_batches, hidden_dim, num_patches] to [num_batches, num_patches, hidden_units].
            mlp1_outputs = keras.ops.transpose(mlp1_outputs, axes=(0, 2, 1))
            # Add skip connection.
            x = mlp1_outputs + inputs
            # Apply layer normalization.
            x_patches = self.normalize(x)
            # Apply mlp2 on each patch independtenly.
            mlp2_outputs = self.mlp2(x_patches)
            # Add skip connection.
            x = x + mlp2_outputs
            return x